In [2]:
import pandas as pd

from tqdm import tqdm

from transformers import ( RobertaTokenizer)
from datasets import Dataset, load_from_disk

from transformers import RobertaTokenizer
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import multiprocessing
import torch

import evaluate
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('../datasets/multi_sard_all_data_notmarked.csv')
df.head()

code CWE-Type DataType
0  \n \n \n #include "std_testcase.h"\n \n #inclu...   CWE114     SARD
1  \n \n \n #include "std_testcase.h"\n \n #inclu...   CWE114     SARD
2  \n \n \n #include "std_testcase.h"\n \n #inclu...   CWE114     SARD
3  \n \n \n #include "std_testcase.h"\n \n #inclu...   CWE114     SARD
4  \n \n \n #include "std_testcase.h"\n \n #inclu...   CWE114     SARD

In [4]:
cwe_counts = df['CWE-Type'].value_counts()
df = df[df['CWE-Type'].isin(cwe_counts[cwe_counts >= 500].index)]
cwe_counts = cwe_counts[cwe_counts >= 500]
df.groupby('CWE-Type').size()

CWE-Type
CWE114     648
CWE121    5703
CWE122    3531
CWE124    1824
CWE126    1332
CWE127    1824
CWE134    3240
CWE190    4860
CWE191    3726
CWE194    1307
CWE195    1308
CWE197     979
CWE252     630
CWE253     684
CWE369     972
CWE400     810
CWE401    1188
CWE427     540
CWE457     616
CWE590     864
CWE606     540
CWE690    1091
CWE761     648
CWE78     5400
CWE789     540
CWE90      540
dtype: int64

In [5]:
# Create a map for CWE type into label
cwe_to_label = {cwe: idx for idx, cwe in enumerate(cwe_counts.index)}

# Create a map for label into CWE type
label_to_cwe = {idx: cwe for cwe, idx in cwe_to_label.items()}

cwe_to_label

{'CWE121': 0,
 'CWE78': 1,
 'CWE190': 2,
 'CWE191': 3,
 'CWE122': 4,
 'CWE134': 5,
 'CWE124': 6,
 'CWE127': 7,
 'CWE126': 8,
 'CWE195': 9,
 'CWE194': 10,
 'CWE401': 11,
 'CWE690': 12,
 'CWE197': 13,
 'CWE369': 14,
 'CWE590': 15,
 'CWE400': 16,
 'CWE253': 17,
 'CWE761': 18,
 'CWE114': 19,
 'CWE252': 20,
 'CWE457': 21,
 'CWE427': 22,
 'CWE789': 23,
 'CWE90': 24,
 'CWE606': 25}

In [6]:
# Add a new column 'label' to the dataframe by mapping 'CWE-Type' to labels
df['label'] = df['CWE-Type'].map(cwe_to_label)
df.head()

code CWE-Type DataType  label
0  \n \n \n #include "std_testcase.h"\n \n #inclu...   CWE114     SARD     19
1  \n \n \n #include "std_testcase.h"\n \n #inclu...   CWE114     SARD     19
2  \n \n \n #include "std_testcase.h"\n \n #inclu...   CWE114     SARD     19
3  \n \n \n #include "std_testcase.h"\n \n #inclu...   CWE114     SARD     19
4  \n \n \n #include "std_testcase.h"\n \n #inclu...   CWE114     SARD     19

In [7]:
from sklearn.model_selection import train_test_split

random_state = 42
df_train, df_test = train_test_split(df, test_size=0.1, random_state=random_state)
df_train, df_test

(                                                    code CWE-Type DataType  \
 30908  \n \n \n #include "std_testcase.h"\n \n #ifnde...   CWE252     SARD   
 20742  \n \n \n #include "std_testcase.h"\n \n #ifnde...   CWE190     SARD   
 21250  \n \n \n #include "std_testcase.h"\n \n #ifnde...   CWE190     SARD   
 19243  \n \n \n #include "std_testcase.h"\n \n #ifnde...   CWE190     SARD   
 28659  \n \n \n #include "std_testcase.h"\n \n #defin...   CWE195     SARD   
 ...                                                  ...      ...      ...   
 11446  \n \n \n #include "std_testcase.h"\n \n #inclu...   CWE124     SARD   
 51993  \n \n \n #include "std_testcase.h"\n \n #inclu...    CWE78     SARD   
 44745  \n \n \n #include "std_testcase.h"\n \n #inclu...   CWE690     SARD   
 860    \n \n \n #include "std_testcase.h"\n \n #ifnde...   CWE121     SARD   
 15957  \n \n \n #include <stdarg.h>\n #include "std_t...   CWE134     SARD   
 
        label  
 30908     20  
 20742      2  
 2

In [8]:
dataset = {}
dataset['text'] = list(df_train['code'])
dataset['label'] = list(df_train['label'])
ds = Dataset.from_dict(dataset)
ds = ds.train_test_split(test_size=0.1)

In [9]:
tokenizer = RobertaTokenizer.from_pretrained('microsoft/codebert-base-mlm', cache_dir="D://hubface_cache")

In [10]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_dataset = ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/36729 [00:00<?, ? examples/s]

Map:   0%|          | 0/4081 [00:00<?, ? examples/s]

In [11]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [12]:
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [13]:
model = AutoModelForSequenceClassification.from_pretrained(
    "microsoft/codebert-base-mlm", num_labels=len(cwe_to_label.keys()), cache_dir="D://hubface_cache"
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base-mlm and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
classifier_weights = {k: v for k, v in model.state_dict().items() if "encoder" in k}
classifier_weights

{'roberta.encoder.layer.0.attention.self.query.weight': tensor([[ 0.1142,  0.0147, -0.0510,  ...,  0.0917,  0.0966, -0.0560],
         [-0.0190,  0.1496,  0.0807,  ...,  0.0638,  0.0821,  0.1236],
         [ 0.0550,  0.0402, -0.0323,  ..., -0.0504, -0.0110,  0.0909],
         ...,
         [-0.1371, -0.0055, -0.0561,  ...,  0.0026,  0.0835, -0.0887],
         [-0.2367,  0.0508,  0.1000,  ...,  0.0855, -0.1373, -0.0198],
         [-0.0708, -0.0662,  0.1019,  ..., -0.1861, -0.0126, -0.1042]]),
 'roberta.encoder.layer.0.attention.self.query.bias': tensor([ 2.2940e-01,  6.8510e-02, -3.6759e-01,  5.0081e-01,  6.9741e-01,
         -2.6569e-01, -1.2555e-01,  2.0404e-01, -1.0488e-01,  3.9673e-01,
          5.4696e-01,  5.9468e-01,  1.9431e-01, -1.6381e-01,  6.0639e-01,
         -2.6454e-01, -2.4744e-01,  1.3515e-01,  2.5058e-02,  9.5239e-03,
         -5.0751e-01,  1.0674e-01, -7.6378e-02, -1.3269e-01, -6.1458e-01,
          1.8137e-01,  7.9792e-02, -4.3903e-01, -2.5528e-02,  3.3939e-01,
      

In [15]:
training_args = TrainingArguments(
    output_dir="trained_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

C:\Users\bowsw\AppData\Roaming\Python\Python312\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\bowsw\AppData\Local\Temp\ipykernel_2472\565366963.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


TrainOutput(global_step=2296, training_loss=0.21496452417107825, metrics={'train_runtime': 918.3066, 'train_samples_per_second': 39.996, 'train_steps_per_second': 2.5, 'total_flos': 9665888370259968.0, 'train_loss': 0.21496452417107825, 'epoch': 1.0})

In [16]:
classifier_weights = {k: v for k, v in model.state_dict().items() if "encoder" in k}
classifier_weights


size_mb = sum(p.numel() * p.element_size() for p in model.parameters()) / (1024 ** 2)
print(f"Model size: {size_mb:.2f} MB")
torch.cuda.empty_cache()

Model size: 475.56 MB


In [17]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

preds = []
for i in tqdm(df_test['code'].values):
    with torch.no_grad():
        inputs = tokenizer(i, return_tensors="pt", truncation=True).to(device)
        logits = model(**inputs).logits
        predicted_class_id = logits.argmax().item()
        preds.append(predicted_class_id)
        

100%|██████████| 4535/4535 [01:04<00:00, 70.83it/s]


In [18]:
model.save_pretrained("categorizing_model_not_marked")
tokenizer.save_pretrained("categorizing_model_not_marked")

('categorizing_model_not_marked\\tokenizer_config.json',
 'categorizing_model_not_marked\\special_tokens_map.json',
 'categorizing_model_not_marked\\vocab.json',
 'categorizing_model_not_marked\\merges.txt',
 'categorizing_model_not_marked\\added_tokens.json')

In [19]:
from sklearn.metrics import classification_report
print(classification_report([i for i in df_test['label'].values], preds))



              precision    recall  f1-score   support

           0       1.00      1.00      1.00       576
           1       1.00      1.00      1.00       538
           2       1.00      1.00      1.00       489
           3       1.00      1.00      1.00       360
           4       1.00      1.00      1.00       339
           5       1.00      1.00      1.00       316
           6       1.00      1.00      1.00       153
           7       0.99      0.99      0.99       195
           8       1.00      1.00      1.00       143
           9       0.98      1.00      0.99       140
          10       0.98      1.00      0.99       133
          11       1.00      1.00      1.00       117
          12       1.00      1.00      1.00       122
          13       1.00      0.98      0.99        87
          14       1.00      1.00      1.00        89
          15       1.00      1.00      1.00        89
          16       1.00      0.98      0.99        89
          17       1.00    

In [20]:
df_test['CWE-Type'].value_counts()

CWE-Type
CWE121    576
CWE78     538
CWE190    489
CWE191    360
CWE122    339
CWE134    316
CWE127    195
CWE124    153
CWE126    143
CWE195    140
CWE194    133
CWE690    122
CWE401    117
CWE590     89
CWE369     89
CWE400     89
CWE197     87
CWE253     76
CWE252     72
CWE114     69
CWE457     67
CWE789     66
CWE761     62
CWE90      51
CWE606     51
CWE427     46
Name: count, dtype: int64

In [21]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

import matplotlib.pyplot as plt
# Generate the confusion matrix
cm = confusion_matrix(df_test['label'], preds)

# Plot the confusion matrix
plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_to_cwe.values(), yticklabels=label_to_cwe.values())
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')

plt.show()